## Exact models: execution example
This notebook includes an example on how to use the exact models and other functions to solve the problem of team formation.

In [1]:
import sys
import pandas as pd
sys.path.insert(0,"..")
from exact_model import *
from general_functions import *

# Reading student data
df = pd.read_csv("../data/student_data_anon.csv",delimiter=";")

# This time we will only be getting Belbin scores for a subset of students
df = df.loc[list(range(1,31)),["student_id","CW","CH","SH","PL","RI","ME","TW","CF"]]

# Do something with the null values (your criterion)
df.fillna(0, inplace=True)

df.head()

,student_id,CW,CH,SH,PL,RI,ME,TW,CF
1,2,15.0,9.0,14.0,15.0,12.0,6.0,9.0,19.0
2,3,17.0,0.0,10.0,26.0,7.0,10.0,7.0,22.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,11.0,16.0,20.0,11.0,13.0,9.0,10.0,10.0
5,6,12.0,14.0,14.0,11.0,11.0,12.0,14.0,12.0


In [2]:
# List of students
students = list(df["student_id"])

# Table of scores into a dictionary
scores_dict = scores_table2dict(df) #from the general_functions module
print(scores_dict)

{'CW': [15.0, 17.0, 0.0, 11.0, 12.0, 7.0, 13.0, 0.0, 11.0, 9.0, 16.0, 13.0, 8.0, 20.0, 10.0, 0.0, 21.0, 18.0, 15.0, 15.0, 15.0, 7.0, 11.0, 13.0, 9.0, 4.0, 11.0, 0.0, 23.0, 13.0], 'CH': [9.0, 0.0, 0.0, 16.0, 14.0, 21.0, 13.0, 0.0, 10.0, 10.0, 10.0, 15.0, 24.0, 11.0, 17.0, 0.0, 21.0, 8.0, 15.0, 13.0, 11.0, 4.0, 14.0, 7.0, 17.0, 7.0, 19.0, 0.0, 7.0, 12.0], 'SH': [14.0, 10.0, 0.0, 20.0, 14.0, 7.0, 14.0, 0.0, 23.0, 20.0, 12.0, 14.0, 16.0, 9.0, 25.0, 0.0, 13.0, 21.0, 19.0, 6.0, 13.0, 12.0, 20.0, 24.0, 16.0, 39.0, 16.0, 0.0, 16.0, 12.0], 'PL': [15.0, 26.0, 0.0, 11.0, 11.0, 10.0, 12.0, 0.0, 13.0, 10.0, 12.0, 11.0, 8.0, 4.0, 12.0, 0.0, 6.0, 4.0, 1.0, 5.0, 13.0, 25.0, 9.0, 17.0, 13.0, 4.0, 10.0, 0.0, 3.0, 13.0], 'RI': [12.0, 7.0, 0.0, 13.0, 11.0, 17.0, 11.0, 0.0, 11.0, 11.0, 12.0, 11.0, 9.0, 9.0, 13.0, 0.0, 3.0, 13.0, 9.0, 10.0, 11.0, 10.0, 9.0, 3.0, 11.0, 19.0, 7.0, 0.0, 12.0, 12.0], 'ME': [6.0, 10.0, 0.0, 9.0, 12.0, 1.0, 8.0, 0.0, 14.0, 4.0, 10.0, 12.0, 7.0, 21.0, 4.0, 0.0, 11.0, 0.0, 12.0, 6.

In [3]:
# Constraint values
MIN_G, MAX_G = 3, 4
no_mates = [{1,3},{7,21}]
compulsory = [{1,2},{10,30}]
restr_num_groups = {3:[1,5]}

# All possible team combinations
combinations = generate_combinations(students, MIN_G, MAX_G)

# Feasible teams
combinations = delete_no_mates(combinations, no_mates)

Using a model from the pywrap module:

In [4]:
resolution_time, result_string, best_teams, best_scores,value = use_pywrap_model(teams = combinations, 
                                                                                 students = students, 
                                                                                 compulsory = compulsory, 
                                                                                 scores_d = scores_dict, 
                                                                                 score_f=1, # Using Belbin
                                                                                 restr_num_groups = restr_num_groups,
                                                                                 sname="SCIP")
print(f"Result type: {result_string} with value {value}")
print(f"Resolution time: {resolution_time} seconds")
print(f"\n Best teams: {best_teams}")

Result type: optimal with value 8.0
Resolution time: 64.68662548065186 seconds

 Best teams: [{12, 5, 7}, {11, 29, 15}, {16, 2, 6, 23}, {26, 9, 18, 3}, {24, 17, 4, 21}, {8, 27, 20, 14}, {10, 19, 13, 30}, {25, 28, 22, 31}]


Using the CP-SAT model:

In [5]:
resolution_time, result_string, best_teams, best_scores,value = use_SAT(teams = combinations, 
                                                                                 students = students, 
                                                                                 compulsory = compulsory, 
                                                                                 scores_d = scores_dict, 
                                                                                 score_f=1, 
                                                                                 restr_num_groups = restr_num_groups)
print(f"Result type: {result_string} with value {value}")
print(f"Resolution time: {resolution_time} seconds")
print(f"\n Best teams: {best_teams}")

Result type: optimal with value 8.0
Resolution time: 13.844407796859741 seconds

 Best teams: [{2, 26, 20}, {16, 17, 18}, {27, 3, 28, 21}, {24, 25, 4, 23}, {29, 19, 5, 22}, {8, 9, 11, 6}, {12, 13, 14, 7}, {10, 31, 30, 15}]
